# EDA of the datasets

## Importing the datasets

In [167]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer # scikit-learn

In [168]:
movie_path=r'data\movies.csv'
rating_path=r'data\ratings.csv'
movie_df=pd.read_csv(movie_path)
rating_df=pd.read_csv(rating_path)

In [169]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [170]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [171]:
print(f"Movie Dataframe Shape {movie_df.shape}")
print(f"Movie Ratings Shape {rating_df.shape}")

Movie Dataframe Shape (9742, 3)
Movie Ratings Shape (100836, 4)


## Checking Null Values

In [172]:
movie_null_sum = movie_df.isnull().sum()
print(movie_null_sum)

movieId    0
title      0
genres     0
dtype: int64


In [173]:
rating_null_sum=rating_df.isnull().sum()
print(rating_null_sum)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


There aren't any null values in both datasets

## Checking the duplicates

In [174]:
duplicates_movies=movie_df.duplicated()
duplicates_movies.head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [175]:
duplicate_rows = movie_df[movie_df.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [movieId, title, genres]
Index: []


In [176]:
duplicate_ratings=rating_df.duplicated()
duplicate_ratings.head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [177]:
duplicate_rows = rating_df[rating_df.duplicated()]
print(duplicate_rows)

Empty DataFrame
Columns: [userId, movieId, rating, timestamp]
Index: []


## Describing the dataframes

In [178]:
movie_df.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [179]:
rating_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# Changing Categorical Values of Genres

In [180]:
movie_df['genres']=movie_df['genres'].str.split("|")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [181]:
movie_df.info()
movie_df['title']=movie_df['title'].astype('string')

movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   string
 2   genres   9742 non-null   object
dtypes: int64(1), object(1), string(1)
memory usage: 228.5+ KB


In [182]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


## Year Extraction

In [183]:
movie_df['year'] = movie_df['title'].str.extract(r'\((\d{4})\)')
movie_df['title'] = movie_df['title'].str.replace(r' \(\d{4}\)', '', regex=True)

movie_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [184]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_genres = pd.DataFrame(mlb.fit_transform(movie_df['genres']), columns=mlb.classes_, index=movie_df.index)
new_df = pd.concat([movie_df[['movieId', 'title', 'year']], df_genres], axis=1)

new_df.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [185]:
new_df.loc[new_df['(no genres listed)']>0]

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
8517,114335,La cravate,1957,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8684,122888,Ben-hur,2016,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tales,2017,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8782,129250,Superfast!,2015,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8836,132084,Let It Be Me,1995,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8902,134861,Trevor Noah: African American,2013,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9033,141131,Guardians,2016,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9053,141866,Green Room,2015,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9070,142456,The Brand New Testament,2015,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9091,143410,Hyena Road,<NA>,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
new_df.loc[new_df['title']=='Black Mirror']

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
9611,176601,Black Mirror,<NA>,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Joining Rating Dataframe and Movie Dataframe

In [187]:
final_df = new_df.merge(rating_df, on='movieId', how='left')
final_df = final_df.drop_duplicates()
final_df.head()

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp
0,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,1.0,4.0,9.649827e+08
1,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,5.0,4.0,8.474350e+08
2,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,7.0,4.5,1.106636e+09
3,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,15.0,2.5,1.510578e+09
4,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,17.0,4.5,1.305696e+09


In [188]:
final_df.shape

(100854, 26)

In [189]:
final_df.isnull().sum()

movieId                0
title                  0
year                  18
(no genres listed)     0
Action                 0
Adventure              0
Animation              0
Children               0
Comedy                 0
Crime                  0
Documentary            0
Drama                  0
Fantasy                0
Film-Noir              0
Horror                 0
IMAX                   0
Musical                0
Mystery                0
Romance                0
Sci-Fi                 0
Thriller               0
War                    0
Western                0
userId                18
rating                18
timestamp             18
dtype: int64

In [193]:
rows_with_nulls = final_df[final_df.isnull().any(axis=1)]
rows_with_nulls

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,userId,rating,timestamp
22820,1076,"Innocents, The",1961,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
49539,2939,Niagara,1953,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN
53555,3338,For All Mankind,1989,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
54467,3456,"Color of Paradise, The (Rang-e khoda)",1999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
60535,4194,I Know Where I'm Going!,1945,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,NaN,NaN,NaN
68396,5721,"Chosen, The",1981,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
71896,6668,"Road Home, The (Wo de fu qin mu qin)",1999,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,NaN,NaN,NaN
72428,6849,Scrooge,1970,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,NaN,NaN,NaN
73354,7020,Proof,1991,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,NaN,NaN,NaN
75450,7792,"Parallax View, The",1974,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,NaN,NaN,NaN


In [196]:
final_df=final_df[~(final_df['userId'].isnull() & final_df['rating'].isnull())]
final_df.isnull().sum()

movieId                0
title                  0
year                  18
(no genres listed)     0
Action                 0
Adventure              0
Animation              0
Children               0
Comedy                 0
Crime                  0
Documentary            0
Drama                  0
Fantasy                0
Film-Noir              0
Horror                 0
IMAX                   0
Musical                0
Mystery                0
Romance                0
Sci-Fi                 0
Thriller               0
War                    0
Western                0
userId                 0
rating                 0
timestamp              0
dtype: int64